In [12]:
# Core
import os
import numpy as np
import pandas as pd

# Visualization (optional)
import matplotlib.pyplot as plt

# Sklearn utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Regression models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# XGBoost
from xgboost import XGBRegressor

# Warnings
import warnings
warnings.filterwarnings("ignore")


In [13]:
# Adjust path if needed
csv_path = "../data/productivity/productivity_daily.csv"

print("Using CSV:", os.path.abspath(csv_path))
print("Exists:", os.path.exists(csv_path))

df = pd.read_csv(csv_path)
print("Shape:", df.shape)

df.head()


Using CSV: d:\imp\certificate\project\Working-Project\EMPLOYEE\Time-Project-Analysis\data\productivity\productivity_daily.csv
Exists: True
Shape: (0, 5)


,user_id,date,score,completed,total


In [14]:
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)


In [15]:
y = df["score"].astype(float)


In [16]:
# Trend index
df["idx"] = np.arange(len(df))

# Day of week (0 = Monday)
df["dow"] = df["date"].dt.weekday

# Avoid division by zero
df["completion_ratio"] = df["completed"] / df["total"].replace(0, 1)

X = df[
    [
        "idx",
        "dow",
        "completed",
        "total",
        "completion_ratio"
    ]
]

X.head()


,idx,dow,completed,total,completion_ratio


In [20]:
print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (0, 5)
y shape: (0,)


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print("Train size:", X_train.shape)
print("Test size :", X_test.shape)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [17]:
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
    "XGBoost": XGBRegressor(
        n_estimators=150,
        learning_rate=0.05,
        max_depth=6,
        random_state=42,
        objective="reg:squarederror"
    )
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    results.append({
        "Model": name,
        "MAE": round(mean_absolute_error(y_test, preds), 2),
        "RMSE": round(np.sqrt(mean_squared_error(y_test, preds)), 2),
        "R2": round(r2_score(y_test, preds), 3)
    })

comparison_df = pd.DataFrame(results).sort_values("MAE")
comparison_df


NameError: name 'X_train' is not defined